In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import joblib
from tqdm import tqdm 

In [2]:
class iResNet(nn.Module):
    def __init__(self, input_shape, full_connect_shape, q, N):
        super(iResNet, self).__init__()
        self.input_shape = input_shape
        self.full_connect_shape = full_connect_shape  # 格式：(Channel, H, W)
        self.q = q
        self.N = N

        # 从 full_connect_shape 提取通道数 C
        self.C, self.H, self.W = full_connect_shape

        # 全连接层：将输入扩展到 (C * H * W)
        self.fc = nn.Linear(np.prod(input_shape), np.prod(full_connect_shape))

        # 创建 q 个 BB
        self.bb_layers = nn.ModuleList()
        for _ in range(q):
            sb_layers = nn.ModuleList()
            for _ in range(N):
                # 动态设置 in_channels 和 out_channels 为 C
                sb_layers.append(nn.Conv2d(self.C, self.C, kernel_size=(1,3), stride=1, padding=(0,1)))
                sb_layers.append(nn.LeakyReLU(0.01))
            self.bb_layers.append(sb_layers)

        # 最后一层：输出通道数为 1
        self.final_conv = nn.Conv2d(self.C, 1, kernel_size=(1,3), stride=1, padding=(0,1))


    def forward(self, x):
        # 全连接层 + Reshape
        x = self.fc(x)
        x = x.view(-1, self.C, self.H, self.W)  # 调整为 (batch_size, C, H, W)

        # 处理每个 BB
        for sb_layers in self.bb_layers:
            bb_input = x
            for layer in sb_layers:
                x = layer(x)
            x = (bb_input + x) * 2

        # 最后一层
        T = self.final_conv(x)
        return T

In [3]:
# 初始化模型
input_shape = (1,)  # 输入形状
full_connect_shape = (128, 1, 128)  # 全连接层目标形状，根据 true_output0 的形状确定
q = 2  # BB 的数量
N = 3  # 每个 BB 中 SB 的数量

model = iResNet(input_shape, full_connect_shape, q, N)

# 打印模型结构
print(model)

iResNet(
  (fc): Linear(in_features=1, out_features=16384, bias=True)
  (bb_layers): ModuleList(
    (0-1): 2 x ModuleList(
      (0): Conv2d(128, 128, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): Conv2d(128, 128, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
      (3): LeakyReLU(negative_slope=0.01)
      (4): Conv2d(128, 128, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
      (5): LeakyReLU(negative_slope=0.01)
    )
  )
  (final_conv): Conv2d(128, 1, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
)


In [4]:
# 确保模型在评估模式（即使未训练）
model.eval()

# 构造输入张量 [1]，形状为 (1,)
# 注意：PyTorch 的输入需要是浮点型张量，且添加 batch 维度（batch_size=1）
input_tensor = torch.tensor([1.0], dtype=torch.float32).unsqueeze(0)  # 形状: (1, 1)

# 不计算梯度，直接前向传播
with torch.no_grad():
    T = model(input_tensor)

# 打印输出
print("输入张量形状:", input_tensor.shape)
print("输出 T 的形状:", T.shape)

print("\n输出 T 的数值示例:")
print(T)  # 打印 u 的第一个样本、第一个通道的部分数值


输入张量形状: torch.Size([1, 1])
输出 T 的形状: torch.Size([1, 1, 1, 128])

输出 T 的数值示例:
tensor([[[[ 1.9287,  1.7928,  0.1930,  1.1660, -2.7588, -1.3195,  1.7819,
            3.3459, -0.8588,  1.5367,  2.4003,  3.3674, -1.1041,  0.8321,
           -3.9467,  0.0680, -0.9672, -0.5930, -1.7762,  1.5503, -0.5277,
            2.3058,  2.7793, -3.2232, -2.2937,  2.6418,  2.8312,  1.6819,
           -0.1138, -1.0177, -0.4491,  4.7019, -2.5032,  3.4072, -2.0444,
           -2.9305, -2.4765, -0.8253, -1.8832, -3.0389,  0.2139, -0.9628,
           -0.7722, -2.3380, -0.4631, -1.0380, -0.5525,  1.3630, -2.0249,
           -3.3796,  1.8974, -0.1139, -0.3960, -2.7922, -0.9499,  1.0262,
            0.4063,  0.9643, -1.9196, -1.1900, -0.0086, -1.1774, -0.8397,
           -0.2820, -2.1832, -2.7541, -0.1457,  3.1886,  0.9606, -4.0544,
            3.3774, -2.0824,  0.0945,  1.4892, -1.7811, -1.4371, -0.5928,
            3.2855, -2.4947,  1.6417, -2.0993,  0.8256,  0.1127, -0.6042,
           -0.8769, -0.3474, -2.120

In [31]:
class Trainer:
    def __init__(self, model, optimizer, criterion_T, device="cuda"):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.criterion_T = criterion_T
        self.device = device
        self.train_losses = []
        # self.val_losses = []
        
    def _train_step(self, inputs, targets_T):
        self.model.train()
        self.optimizer.zero_grad()
        
        # 前向传播
        preds_T = self.model(inputs)
        
        # 计算损失
        loss_T = self.criterion_T(preds_T, targets_T)
        total_loss = loss_T
        
        # 反向传播
        total_loss.backward()
        self.optimizer.step()
        
        return total_loss.item()
    
    def _val_step(self, inputs, targets_T):
        self.model.eval()
        with torch.no_grad():
            preds_T = self.model(inputs)
            loss_T = self.criterion_T(preds_T, targets_T)
            return (loss_T).item()
    
    def fit(self, train_loader,  num_epochs=50, early_stop_patience=5, save_path="best_model.pth"):
        best_train_loss = float('inf')  # 用于保存最佳训练损失
        early_stop_counter = 0
        
        for epoch in range(num_epochs):
            # 训练阶段
            train_loss = 0.0
            for inputs, targets_T in tqdm(train_loader, desc=f"Epoch {epoch+1}", position=0, leave=True, ncols=100):
                inputs = inputs.to(self.device)
                targets_T = targets_T.unsqueeze(1).to(self.device)
                train_loss += self._train_step(inputs, targets_T)
            
            # # 验证阶段
            # val_loss = 0.0
            # for inputs, (targets_T) in val_loader:
            #     inputs = inputs.to(self.device)
            #     targets_T = targets_T.unsqueeze(1).to(self.device)
            #     val_loss += self._val_step(inputs, targets_T)
            
            # 计算平均损失
            train_loss /= len(train_loader)
            # val_loss /= len(val_loader)
            self.train_losses.append(train_loss)
            # self.val_losses.append(val_loss)
            
            # 保存最佳模型
            if train_loss < best_train_loss:
                best_train_loss = train_loss
                torch.save(self.model.state_dict(), save_path)  # 保存模型权重
                early_stop_counter = 0
            else:
                early_stop_counter += 1
                if early_stop_counter >= early_stop_patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
                
                print(f"Epoch {epoch+1}/{num_epochs} | "
                      f"Train Loss: {train_loss:.4f} ")
    
    def plot_loss(self):
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, 5))
        plt.plot(self.train_losses, label='Train Loss')
        # plt.plot(self.val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

In [32]:
# 定义损失函数和优化器
criterion_T = nn.MSELoss()  # 用于 targets_u 的损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [33]:
# 加载数据
u_train_np = joblib.load('u_train').astype(np.float32)
T_train_np = joblib.load('T_train').astype(np.float32)

u_train = torch.from_numpy(u_train_np).unsqueeze(1)
T_train = torch.from_numpy(T_train_np).unsqueeze(1)

train_dataset = TensorDataset(u_train, T_train)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)

In [34]:
trainer = Trainer(model, optimizer, criterion_T, device="cuda")


In [37]:
trainer.fit(train_loader, num_epochs=1000, early_stop_patience=100, save_path="best_model.pth")

Epoch 7: 100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00, 157.08it/s]


Epoch 7/1000 | Train Loss: 0.0003 


Epoch 566: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 175.61it/s]


Epoch 566/1000 | Train Loss: 0.0000 


Epoch 567: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 174.36it/s]


...

Epoch 998/1000 | Train Loss: 0.0000 


Epoch 999: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 152.03it/s]


Epoch 999/1000 | Train Loss: 0.0000 


Epoch 1000: 100%|████████████████████████████████████████████████████| 1/1 [00:00<00:00, 154.19it/s]


Epoch 1000/1000 | Train Loss: 0.0000 
